In [1]:
import os
import numpy as np
# from nltk.corpus import stopwords
from string import punctuation
from gensim.models import word2vec

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
np.random.seed(42)

In [ ]:
# https://newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3

In [ ]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

In [4]:
!python -m spacy download en_core_web_sm

2023-07-11 08:23:29.551372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 08:23:31.358225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-11 08:23:33.929074: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 08:23:33.929552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading the training and test data sets

In [8]:
train_data = "/content/drive/MyDrive/DSCP Deep Learning/NLP Projects/Session_2/TrainData.txt"
test_data = "/content/drive/MyDrive/DSCP Deep Learning/NLP Projects/Session_2/TestData.txt"

In [9]:
X, y = [], []
with open(train_data, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        X.append(text.split())
        y.append(label)
X, y = np.array(X), np.array(y)
print ("total examples %s" % len(y))

total examples 5485


In [10]:
print(X[0])

['champion', 'products', 'ch', 'approves', 'stock', 'split', 'champion', 'products', 'inc', 'said', 'its', 'board', 'of', 'directors', 'approved', 'a', 'two', 'for', 'one', 'stock', 'split', 'of', 'its', 'common', 'shares', 'for', 'shareholders', 'of', 'record', 'as', 'of', 'april', 'the', 'company', 'also', 'said', 'its', 'board', 'voted', 'to', 'recommend', 'to', 'shareholders', 'at', 'the', 'annual', 'meeting', 'april', 'an', 'increase', 'in', 'the', 'authorized', 'capital', 'stock', 'from', 'five', 'mln', 'to', 'mln', 'shares', 'reuter']


In [11]:
y[0]

'earn'

In [12]:
X_test, y_test = [], []
with open(test_data, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        X_test.append(text.split())
        y_test.append(label)
X_test, y_test = np.array(X_test), np.array(y_test)
print ("total examples %s" % len(y_test))

total examples 2189


In [13]:
X.shape, X_test.shape

((5485,), (2189,))

In [14]:
np.unique(y, return_counts=True)

(array(['acq', 'crude', 'earn', 'grain', 'interest', 'money-fx', 'ship',
        'trade'], dtype='<U8'),
 array([1596,  253, 2840,   41,  190,  206,  108,  251]))

In [15]:
X[0],

(['champion',
  'products',
  'ch',
  'approves',
  'stock',
  'split',
  'champion',
  'products',
  'inc',
  'said',
  'its',
  'board',
  'of',
  'directors',
  'approved',
  'a',
  'two',
  'for',
  'one',
  'stock',
  'split',
  'of',
  'its',
  'common',
  'shares',
  'for',
  'shareholders',
  'of',
  'record',
  'as',
  'of',
  'april',
  'the',
  'company',
  'also',
  'said',
  'its',
  'board',
  'voted',
  'to',
  'recommend',
  'to',
  'shareholders',
  'at',
  'the',
  'annual',
  'meeting',
  'april',
  'an',
  'increase',
  'in',
  'the',
  'authorized',
  'capital',
  'stock',
  'from',
  'five',
  'mln',
  'to',
  'mln',
  'shares',
  'reuter'],)

In [17]:
y[561]

'earn'

First, reconstructring the string text for our vectorizers to work

## Using our word embeddings approach

 We have two options here:

1. Use pre-trained word vectors (Glove)

2. Train our own vectors

We'll explore both

### Loading the pre-trained GloVe word vectors

In [18]:
import gensim

In [19]:
import gensim.downloader as api

In [20]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [21]:
import gensim.downloader as api
wv = api.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [22]:
wv.most_similar('happy')

[('birthday', 0.9478021860122681),
 ('day', 0.8840912580490112),
 ('love', 0.8704721331596375),
 ('thank', 0.8623552918434143),
 ('wish', 0.8603641390800476),
 ('miss', 0.8560664057731628),
 ('bday', 0.847211480140686),
 ('merry', 0.8461399078369141),
 ('welcome', 0.8427106142044067),
 ('hope', 0.8396721482276917)]

In [23]:
wv.vector_size

50

### Sentence vector by averaging word vectors

In [24]:
import string
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [25]:
def sent_vec(sent,w2v):
    vector_size = w2v.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in w2v:
            ctr += 1
            wv_res += w2v[w]
    wv_res = wv_res/ctr
    return wv_res

In [26]:
sent_vec("I am happy",wv)

array([ 0.11221238,  0.08086875,  0.07783688, -0.038995  , -0.18682502,
        0.17493849, -0.42813626,  0.18419874, -0.13961012, -0.1894935 ,
       -0.18459575, -0.39523976, -2.99103752, -0.0456085 , -0.02529901,
        0.33934125, -0.37086751, -0.10923225, -0.102813  ,  0.85966501,
       -0.24663988, -0.36515624,  0.20198388,  0.03161625,  0.02450538,
       -1.66976251, -0.04163662, -0.0357965 ,  0.26119274,  0.57394749,
        0.04942275, -0.32102951,  0.13673   , -0.03791625,  0.52639925,
        0.50934474, -0.22326875, -0.27457125, -0.59217638,  0.28203475,
       -2.33953753,  0.19693125,  0.2576075 , -0.14676042,  0.259737  ,
        0.10288475, -0.10105875, -0.08618125,  0.12835125, -0.84626376])

In [27]:
print(X[0])

['champion', 'products', 'ch', 'approves', 'stock', 'split', 'champion', 'products', 'inc', 'said', 'its', 'board', 'of', 'directors', 'approved', 'a', 'two', 'for', 'one', 'stock', 'split', 'of', 'its', 'common', 'shares', 'for', 'shareholders', 'of', 'record', 'as', 'of', 'april', 'the', 'company', 'also', 'said', 'its', 'board', 'voted', 'to', 'recommend', 'to', 'shareholders', 'at', 'the', 'annual', 'meeting', 'april', 'an', 'increase', 'in', 'the', 'authorized', 'capital', 'stock', 'from', 'five', 'mln', 'to', 'mln', 'shares', 'reuter']


In [28]:
train_doc_vecs = []
for doc in X:
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec(doc_words,wv))

In [29]:
train_doc_vecs[1]

array([-0.01262836,  0.22475482, -0.30508161, -0.03908617,  0.17813363,
       -0.00792802,  0.15130636, -0.44985048,  0.30153181, -0.46396028,
        0.09798007, -0.14816084, -2.66282607,  0.16549645,  0.32899351,
        0.10251539,  0.20031406,  0.14764548,  0.13225703, -0.09761118,
       -0.20834011, -0.33095987,  0.12465344, -0.04685648, -0.44481558,
        0.35707202,  0.0933768 ,  0.05449308,  0.03042676,  0.09920227,
        0.13932405, -0.17060542, -0.05331427, -0.19822329,  0.60086589,
       -0.13559197, -0.11600278,  0.32340528,  0.09320537, -0.22914279,
       -0.22424682,  0.25530006,  0.08548902, -0.1742663 ,  0.11205718,
       -0.38357651, -0.0615446 , -0.06995826, -0.20953643,  0.24869365])

In [30]:
test_doc_vecs = []
for doc in X_test:
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec(doc_words,wv))

### Building a predictive model on the averaged word vectors

#### Using a 'simple' logistic regression

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [32]:
train_doc_vecs[0]

array([ 0.00676942,  0.27081164,  0.06841652,  0.05023073,  0.28684623,
       -0.15695542,  0.08939592, -0.08921531,  0.41441803, -0.65612471,
       -0.058115  , -0.22393934, -2.48950386,  0.24976179,  0.48492829,
        0.07893562,  0.02060031,  0.20531392,  0.22678279, -0.07675518,
       -0.21046273, -0.40649087,  0.12458603, -0.16542174, -0.36236679,
        0.61257026,  0.20957085, -0.10106019,  0.0083548 , -0.01088134,
        0.33248252, -0.05363525, -0.1859415 , -0.05638189,  0.69560773,
       -0.1910173 ,  0.05431459,  0.30120048,  0.22197218, -0.23174342,
       -0.34453248,  0.03348233, -0.00454458,  0.08305587,  0.31787176,
       -0.58798182, -0.20719374, -0.29887145, -0.31579711,  0.16984145])

In [35]:
type(train_doc_vecs)

list

In [33]:
logreg = LogisticRegression()
logreg.fit(train_doc_vecs,y)

LogisticRegression()

In [34]:
pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))

Train accuracy:  0.9112123974475843
Test accuracy:  0.9081772498857926


### Training our own wordvectors on the data
We'll create a combined text file to train our word vectors - more data is better. Although in this case we would still have just 7.7K instances to learn from.

In [36]:
X_comb = np.concatenate([X,X_test])
len(X_comb)

7674

In [37]:
print(X_comb[5])

['dean', 'foods', 'df', 'sees', 'strong', 'th', 'qtr', 'earnings', 'dean', 'foods', 'co', 'expects', 'earnings', 'for', 'the', 'fourth', 'quarter', 'ending', 'may', 'to', 'exceed', 'those', 'of', 'the', 'same', 'year', 'ago', 'period', 'chairman', 'kenneth', 'douglas', 'told', 'analysts', 'in', 'the', 'fiscal', 'fourth', 'quarter', 'the', 'food', 'processor', 'reported', 'earnings', 'of', 'cts', 'a', 'share', 'douglas', 'also', 'said', 'the', 'year', 's', 'sales', 'should', 'exceed', 'billion', 'dlrs', 'up', 'from', 'billion', 'dlrs', 'the', 'prior', 'year', 'he', 'repeated', 'an', 'earlier', 'projection', 'that', 'third', 'quarter', 'earnings', 'will', 'probably', 'be', 'off', 'slightly', 'from', 'last', 'year', 's', 'cts', 'a', 'share', 'falling', 'in', 'the', 'range', 'of', 'cts', 'to', 'cts', 'a', 'share', 'douglas', 'said', 'it', 'was', 'too', 'early', 'to', 'project', 'whether', 'the', 'anticipated', 'fourth', 'quarter', 'performance', 'would', 'be', 'enough', 'for', 'us', 'to', 

In [38]:
w2v = word2vec.Word2Vec(X_comb, window=2, min_count=2)

In [39]:
w2v.wv.most_similar("one", topn=5)

[('every', 0.7473658323287964),
 ('three', 0.7439424991607666),
 ('two', 0.704211413860321),
 ('each', 0.6976823210716248),
 ('four', 0.6836116909980774)]

In [40]:
w2v.wv['apple']

array([-3.18738329e-03,  1.41273933e-02, -1.39298954e-03, -1.03698811e-02,
        7.62150681e-04, -5.62057979e-02,  2.14620940e-02,  7.07742274e-02,
       -2.86698099e-02, -2.12302115e-02, -2.80286127e-04, -3.41123268e-02,
       -2.15377100e-02, -3.28771933e-03, -3.09268804e-03, -2.28220280e-02,
        1.05361994e-02,  1.16836317e-02, -9.15678218e-03, -5.21176234e-02,
       -1.77819293e-03,  1.40434655e-03,  3.68775390e-02, -7.76962005e-03,
        8.00761022e-03,  4.12663724e-03, -2.86767632e-02, -2.03002971e-02,
       -1.66551881e-02, -6.93050306e-03,  2.79274285e-02, -7.71680009e-03,
        4.45600972e-02,  8.40516575e-03, -1.43702347e-02,  3.19103785e-02,
        7.57041344e-05, -1.30794439e-02, -6.47340063e-03, -4.60468903e-02,
       -7.19877798e-03, -2.29845643e-02,  1.01817017e-02,  3.64182587e-03,
       -6.18332252e-03,  1.13989357e-02, -1.16512943e-02,  7.26122875e-04,
        1.09978309e-02,  1.70796197e-02, -2.52499282e-02, -1.92917362e-02,
        9.67594213e-04,  

#### Getting the sentence vectors for the test and train sets

In [41]:
train_doc_vecs = []
for doc in X:
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec(doc_words,w2v.wv))

In [42]:
test_doc_vecs = []
for doc in X_test:
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec(doc_words,w2v.wv))

#### Logistic regression

In [43]:
def fit_predict(algo):
  algo.fit(train_doc_vecs,y)
  pred_train_ys = algo.predict(train_doc_vecs)
  pred_test_ys = algo.predict(test_doc_vecs)
  print("Train accuracy: ", accuracy_score(pred_train_ys, y))
  print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))

In [44]:
from sklearn.linear_model import LogisticRegression
algo = LogisticRegression()
fit_predict(algo)

Train accuracy:  0.9485870556061987
Test accuracy:  0.9520328917313842


In [45]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
fit_predict(rf)

Train accuracy:  0.9987237921604376
Test accuracy:  0.9447236180904522
